### character Level text generator (modified)

This notebook will be based on the previous notebook. The goal is to improve the accuracy of the text generation model as compared to the previously created model.

Code cells from the previous notebook will remain the same. We are going to make few changes.


_"The model will be fed with a word and will predict what the next character in the sentence will be. This process will repeat itself until we generate a sentence of our desired length"._

### Imports


In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable

import os, time, pickle, string, random
import numpy as np

torch.__version__

'1.9.0+cu102'

### Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Data
We are going to use the dataset that I've downloaded [here](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt) we are then going to load it from google drive and the steps will be as follows:


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_path = "/content/drive/My Drive/NLP Data/text-gen/input.txt"
os.path.exists(file_path)

True

### Loading the dataset.

First, we'll define the sentences that we want our model to output the first few characters. Our dataset is a text file containing Shakespeare's plays or books that we will extract sequence of chars to use as input to our model. Then our model will learn how to complete sentences like "Shakespeare would do".

### SEEDS


In [5]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
np.random.seed(SEED)
random.seed(SEED)

### Loading the data

In [6]:
def load_text_data(filename, init_dialog=False):
  """
  Setting init_dialog = True will remove lines where the character who is going to speak is indicate
  """
  sentences = []
  with open(filename, 'r') as reader:
    for line in reader:
      if init_dialog or ":" not in line:
        sentences.append(line[:-1])

  return sentences
  

In [7]:
sentences = load_text_data(file_path)
print('Number of sentences: ', len(sentences))
print(sentences[:20])

Number of sentences:  29723
['Before we proceed any further, hear me speak.', '', 'Speak, speak.', '', 'You are all resolved rather to die than to famish?', '', 'Resolved. resolved.', '', 'First, you know Caius Marcius is chief enemy to the people.', '', "We know't, we know't.", '', "Let us kill him, and we'll have corn at our own price.", "Is't a verdict?", '', '', 'One word, good citizens.', '', 'We are accounted poor citizens, the patricians good.', 'would yield us but the superfluity, while it were']


### Data Cleaning
We will convert to lowercase the text and remove non alphanumeric chracters (a parameter configuration).

In [8]:
def clean_text(sentences, alpha=False):
  if alpha:
    # Remove non alphabetic character
    cleaned_text = [''.join([t.lower() for t in text if t.isalpha() or t.isspace()])
      for text in sentences]
  else:
    cleaned_text = [t.lower() for t in sentences]
  
  return [t for t in cleaned_text if t!='']
  

In [9]:
# Clean the sentences
sentences = clean_text(sentences)
# Join all the sentences in a one long string
sentences = ' '.join(sentences)
print('Number of characters: ', len(sentences))
print(sentences[:100])

Number of characters:  894876
before we proceed any further, hear me speak. speak, speak. you are all resolved rather to die than 


### Creating the dictionary

Now we'll create a dictionary out of all the characters that we have in the sentences and map them to an integer. This will allow us to convert our input characters to their respective integers (char2int) and viceversa (int2char).

In [10]:
class CharVocab:
  def __init__(self, type_vocab,
               pad_token = "<pad>",
               eos_token = "<eos>",
               unk_token = "<unk>"
               ):
    self.type = type_vocab
    self.int2char = []
    if pad_token !=None:
      self.int2char += [pad_token]
    if eos_token !=None:
      self.int2char += [eos_token]
    if unk_token !=None: 
      self.int2char += [unk_token]

    self.char2int = {}
  
  def __call__(self, text):
    chars = set(''.join(text))
    self.int2char += list(chars)
    self.char2int = {char: ind for ind, char in enumerate(self.int2char)}

In [11]:

vocab = CharVocab('char', None, None,'<unk>')
vocab(sentences)
print('Length of vocabulary: ', len(vocab.int2char))
print('Int to Char: ', vocab.int2char)
print('Char to Int: ', vocab.char2int)

Length of vocabulary:  38
Int to Char:  ['<unk>', 'u', 'z', ';', 'q', 'e', 'b', 'f', ',', '.', 'r', 'h', 's', "'", '$', 'y', 'm', 'a', 't', '-', '&', 'x', 'n', 'p', 'w', '?', 'd', 'k', 'i', ' ', 'o', 'c', 'g', 'j', 'v', '!', 'l', '3']
Char to Int:  {'<unk>': 0, 'u': 1, 'z': 2, ';': 3, 'q': 4, 'e': 5, 'b': 6, 'f': 7, ',': 8, '.': 9, 'r': 10, 'h': 11, 's': 12, "'": 13, '$': 14, 'y': 15, 'm': 16, 'a': 17, 't': 18, '-': 19, '&': 20, 'x': 21, 'n': 22, 'p': 23, 'w': 24, '?': 25, 'd': 26, 'k': 27, 'i': 28, ' ': 29, 'o': 30, 'c': 31, 'g': 32, 'j': 33, 'v': 34, '!': 35, 'l': 36, '3': 37}


We are then going to save the dictionary.

In [12]:
with open( "/content/drive/My Drive/NLP Data/text-gen/char_dict.plk", "wb") as f:
  pickle.dump(vocab.char2int, f)

with open( "/content/drive/My Drive/NLP Data/text-gen/int_dict.plk", "wb") as f:
  pickle.dump(vocab.int2char, f)

print("Done")


Done


### Creating the input data and labels for training.

As we're going to predict the next character in the sequence at each time step, we'll have to divide each sentence into

* **Input data:** The last input character should be excluded as it does not need to be fed into the model
* **Target/Ground Truth Label:** One time-step ahead of the Input data as this will be the "correct answer" for the model at each time step corresponding to the input data



In [13]:
def one_hot_encode(indices, dict_size):
  features = np.eye(dict_size, dtype="float32")[indices.flatten()]
  features = features.reshape((*indices.shape, dict_size))
  return features

def encode_text(input_text, vocab, one_hot=False):
  ''' Encode the input_text replacing the char by its integer number based on the dictionary vocab'''
  output = [vocab.char2int.get(character,0) for character in input_text]
  if one_hot:
    dict_size = len(vocab.char2int)
    return one_hot_encode(output, dict_size)
  else:
    return np.array(output)


Now, we can encode our text, replacing every character by the integer value in the dictionary. When we have our dataset unified and prepared, we should do a quick check and see an example of the data our model will be trained on. This is generally a good idea as it allows you to see how each of the further processing steps affects the reviews and it also ensures that the data has been loaded correctly.

In [14]:
train_data = encode_text(sentences, vocab, one_hot=False)

# Create the input sequence, from 0 to len-1
input_seq =train_data[:-1]

# Create the target sequence, from 1 to len. It is right-shifted one place

target_seq=train_data[1:]
print('\nOriginal text:')
print(sentences[:100])
print('\nEncoded text:')
print(train_data[:100])
print('\nInput sequence:')
print(input_seq[:100])
print('\nTarget sequence:')
print(target_seq[:100])



Original text:
before we proceed any further, hear me speak. speak, speak. you are all resolved rather to die than 

Encoded text:
[ 6  5  7 30 10  5 29 24  5 29 23 10 30 31  5  5 26 29 17 22 15 29  7  1
 10 18 11  5 10  8 29 11  5 17 10 29 16  5 29 12 23  5 17 27  9 29 12 23
  5 17 27  8 29 12 23  5 17 27  9 29 15 30  1 29 17 10  5 29 17 36 36 29
 10  5 12 30 36 34  5 26 29 10 17 18 11  5 10 29 18 30 29 26 28  5 29 18
 11 17 22 29]

Input sequence:
[ 6  5  7 30 10  5 29 24  5 29 23 10 30 31  5  5 26 29 17 22 15 29  7  1
 10 18 11  5 10  8 29 11  5 17 10 29 16  5 29 12 23  5 17 27  9 29 12 23
  5 17 27  8 29 12 23  5 17 27  9 29 15 30  1 29 17 10  5 29 17 36 36 29
 10  5 12 30 36 34  5 26 29 10 17 18 11  5 10 29 18 30 29 26 28  5 29 18
 11 17 22 29]

Target sequence:
[ 5  7 30 10  5 29 24  5 29 23 10 30 31  5  5 26 29 17 22 15 29  7  1 10
 18 11  5 10  8 29 11  5 17 10 29 16  5 29 12 23  5 17 27  9 29 12 23  5
 17 27  8 29 12 23  5 17 27  9 29 15 30  1 29 17 10  5 29 17 36 36 29 10
  

Now we can save our encoded dataset to a file, so we can restore it whenever it is necessary. It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, we will save the dataset as a pickle object, it is the array containing the whole dataset encoded as an integer value for every character.

In [15]:
with open( "/content/drive/My Drive/NLP Data/text-gen/input_data.plk", "wb") as f:
  pickle.dump(train_data, f)

print("Done")

Done


Lets check our one-hot-encode function that we will use later during the training phase:

In [16]:
print('Encoded characters: ',train_data[100:107])
print('One-hot-encoded characters: ',one_hot_encode(train_data[100:107], 38))


Encoded characters:  [18 30 29  7 17 16 28]
One-hot-encoded characters:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


### Creating a batch data generator

When training on the dataset, we need to extract a batch size examples from the inputs and targets, forward and backward the RNN and then repite the iteration with another batch size examples. A batch generator will help us to extract a batch size examples from our datasets.

In [17]:
def batch_generator_sequence(
    features_seq, label_seq, batch_size, seq_len
  ):
  """Generator function that yields batches of data (input and target)

  Args:
      features_seq: sequence of chracters, feature of our model.
      label_seq: sequence of chracters, the target label of our model
      batch_size (int): number of examples (in this case, sentences) per batch.
      seq_len (int): maximum length of the output tensor.

  Yields:
      x_epoch: sequence of features for the epoch
      y_epoch: sequence of labels for the epoch
  """
  num_batches = len(features_seq) //(batch_size * seq_len)
  if num_batches == 0:
    raise ValueError("No batches created. Use smaller batch size or sequence length.")

  # calculate effective length of text to use
  rounded_len = num_batches * batch_size * seq_len
  # Reshape the features matrix in batch size x num_batches * seq_len
  x = np.reshape(features_seq[: rounded_len], [batch_size, num_batches * seq_len])
  # Reshape the target matrix in batch size x num_batches * seq_len
  y = np.reshape(label_seq[: rounded_len], [batch_size, num_batches * seq_len])

  epoch = 0
  while True:
    # roll so that no need to reset rnn states over epochs
    x_epoch = np.split(np.roll(x, -epoch, axis=0), num_batches, axis=1)
    y_epoch = np.split(np.roll(y, -epoch, axis=0), num_batches, axis=1)
    for batch in range(num_batches):
        yield x_epoch[batch], y_epoch[batch]
    epoch += 1


### Define RNN model

The model is very simple:
* An LSTM layer to encode the input (there is no need for an embedding layer because the data is one-hot-encoded)
* A dropout layer to reduce overfitting
* The decoder, a fully connected layer mapping to a vocabulary size outputs

The output provides the probability of every item in the vocabulary to be the next char.


In [18]:
class RNNModel(nn.Module):
  def __init__(self, vocab_size, embedding_size,
               hidden_dim, n_layers, dropout=.2):
    super(RNNModel, self).__init__()

    self.hidden_dim = hidden_dim
    self.embedding_size = embedding_size
    self.n_layers = n_layers
    self.vocab_size = vocab_size
    self.dropout = dropout
    self.char2int = None
    self.int2char = None

    self.dropout = nn.Dropout(dropout)
    self.rnn = nn.LSTM(embedding_size, hidden_dim, 
                        n_layers, dropout=dropout, batch_first = True)
    self.decoder = nn.Linear(hidden_dim, vocab_size)

  def forward(self, x, state):
    # input shape: [batch_size, seq_len, embedding_size]
    out, state = self.rnn(x, state)
    # out shape: [batch_size, seq_len, rnn_size]
    # hidden shape: [2, num_layers, batch_size, rnn_size]
    out = self.dropout(out)
    out = out.contiguous().view(-1, self.hidden_dim)
    logits = self.decoder(out)
    # output shape: [seq_len * batch_size, vocab_size]
    return logits, state

  def init_state(self, device, batch_size=1):
    """
    initialises rnn states.
    """
    return (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device),
                torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device))
    
  def predict(self, input):
    # input shape: [seq_len, batch_size]
    logits, hidden = self.forward(input)
    # logits shape: [seq_len * batch_size, vocab_size]
    # hidden shape: [2, num_layers, batch_size, rnn_size]
    probs = F.softmax(logits)
    # shape: [seq_len * batch_size, vocab_size]
    probs = probs.view(input.size(0), input.size(1), probs.size(1))
    # output shape: [seq_len, batch_size, vocab_size]
    return probs, hidden


### Training the model
During training:
* In every epoch get the next batch data, move the tensors to the device, call the model (Forward pass), calculate the loss function, get the gradients and update the weights.

In [19]:
def train_main(model, optimizer, loss_fn,
               batch_data, num_batches, val_batches, 
               batch_size, seq_len, n_epochs,
               clip_norm, device
  ):
  # Training Run
  for epoch in range(1, n_epochs + 1):
    # Store the loss in every batch iteration
    #epoch_losses = torch.Tensor(num_batches)
    epoch_losses = []
    # Init the hidden state
    hidden = model.init_state(device, batch_size)
    # Train all the batches in every epoch
    print("Epoch {}/{}".format(epoch, n_epochs+1))
    for i in range(num_batches-val_batches):
      # Get the next batch data for input and target
      input_batch, target_batch = next(batch_data)
      # Onr hot encode the input data
      input_batch = one_hot_encode(input_batch, model.vocab_size)
      # Tranform to tensor
      input_data = torch.from_numpy(input_batch)
      target_data = torch.from_numpy(target_batch)
      # Create a new variable for the hidden state, necessary to calculate the gradients
      hidden = tuple(([Variable(var.data) for var in hidden]))
      # Move the input data to the device
      input_data = input_data.to(device)
      #print('Input shape: ', input_data.shape)
      #print('Hidden shape: ', hidden[0].shape, hidden[1].shape)
      # Set the model to train and prepare the gradients
      model.train()
      optimizer.zero_grad() # Clears existing gradients from previous epoch
      # Pass Fordward the RNN
      output, hidden = model(input_data, hidden)
      #print('Output shape: ', output.shape)
      output = output.to(device)
      #print('Output shape: ', output.shape)
      #print('Target shape; ', target_data.shape)
      # Move the target data to the device
      target_data = target_data.to(device)
      #print('Target shape; ', target_data.shape)
      target_data = torch.reshape(target_data, (batch_size*seq_len,))
      #print('Target shape; ', target_data.shape)
      loss = loss_fn(output, target_data.view(batch_size*seq_len))
      #print(loss)
      # Save the loss
      #epoch_losses[i] = loss.item() #data[0]
      epoch_losses.append(loss.item()) #data[0]
  
      loss.backward() # Does backpropagation and calculates gradients
      # clip gradient norm
      nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
      
      optimizer.step() # Updates the weights accordingly

    # Now, when epoch is finished, evaluate the model on validation data
    model.eval()
    val_hidden = model.init_state(device, batch_size)
    val_losses = []
    print("Val Epoch {}/{}".format(epoch, n_epochs+1))
    for i in range(val_batches):
      # Get the next batch data for input and target
      input_batch, target_batch = next(batch_data)
      # Onr hot encode the input data
      input_batch = one_hot_encode(input_batch, model.vocab_size)
      # Tranform to tensor
      input_data = torch.from_numpy(input_batch)
      target_data = torch.from_numpy(target_batch)
      # Create a new variable for the hidden state, necessary to calculate the gradients
      hidden = tuple(([Variable(var.data) for var in val_hidden]))
      # Move the input data to the device
      input_data = input_data.to(device)
      # Pass Fordward the RNN
      output, hidden = model(input_data, hidden)
      #print('Output shape: ', output.shape)
      output = output.to(device)
      #print('Output shape: ', output.shape)
      #print('Target shape; ', target_data.shape)
      # Move the target data to the device
      target_data = target_data.to(device)
      #print('Target shape; ', target_data.shape)
      target_data = torch.reshape(target_data, (batch_size*seq_len,))
      #print('Target shape; ', target_data.shape)
      loss = loss_fn(output, target_data.view(batch_size*seq_len))
      #print(loss)
      # Save the loss
      val_losses.append(loss.item()) #data[0]

    model.train()                  
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Train Loss: {:.4f}".format(np.mean(epoch_losses)), end=' ')
    print("Val Loss: {:.4f}".format(np.mean(val_losses)))
    print()

    if epoch != n_epochs:
      print("> Next epoch")
  return epoch_losses


After defining the model above, we'll have to instantiate the model with the relevant parameters and define our hyperparamters as well. The hyperparameters we're defining below are:

* n_epochs: Number of Epochs --> This refers to the number of times our model will go through the entire training dataset
* lr: Learning Rate --> This affects the rate at which our model updates the weights in the cells each time backpropogation is done
  * A smaller learning rate means that the model changes the values of the weight with a smaller magnitude
  * A larger learning rate means that the weights are updated to a larger extent for each time step
* batch_size: Number of examples to train on every train step
maxlen: Length of the input sequence of char
* embedding_size: the vocab size because the input feature is one-hot-encoded
* hidden_dim: the number of hidden units in our LSTM module
* n_layers: number of layers of our LSTM module

In [20]:
# Define hyperparameters for training
n_epochs = 20
# lr=0.001
batch_size=16
maxlen=64
clip_norm=5
val_fraction = 0.1

# Define hypeparameters of the model
hidden_dim = 512 #64
n_layers = 1
embedding_size=len(vocab.char2int)
dict_size = len(vocab.char2int)
drop_rate = 0.5

Optimizer and loss functions

In [21]:
model = RNNModel(dict_size,embedding_size, hidden_dim, n_layers)
model

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


RNNModel(
  (dropout): Dropout(p=0.2, inplace=False)
  (rnn): LSTM(38, 512, batch_first=True, dropout=0.2)
  (decoder): Linear(in_features=512, out_features=38, bias=True)
)

Counting model parameters.

In [22]:
def count_trainable_params(model):
  n_t_params =sum(p.numel() for p in model.parameters() if p.requires_grad == True)
  return n_t_params

print(f'The model has {count_trainable_params(model):,} trainable parameters')


The model has 1,149,990 trainable parameters


In [23]:
model = model.to(device)
print(model)

RNNModel(
  (dropout): Dropout(p=0.2, inplace=False)
  (rnn): LSTM(38, 512, batch_first=True, dropout=0.2)
  (decoder): Linear(in_features=512, out_features=38, bias=True)
)


In [24]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [25]:

# Calculate the number of batches to train
num_batches = len(input_seq) // (batch_size*maxlen)
val_batches = int(num_batches*val_fraction)

# Create the batch data generator
batch_data = batch_generator_sequence(input_seq, target_seq, batch_size, maxlen)
losses = train_main(model, optimizer, criterion, batch_data, num_batches, val_batches, batch_size, 
                    maxlen, n_epochs, clip_norm, device)


Epoch 1/21
Val Epoch 1/21
Epoch: 1/20............. Train Loss: 2.2613 Val Loss: 1.9667

> Next epoch
Epoch 2/21
Val Epoch 2/21
Epoch: 2/20............. Train Loss: 1.8178 Val Loss: 1.7450

> Next epoch
Epoch 3/21
Val Epoch 3/21
Epoch: 3/20............. Train Loss: 1.6543 Val Loss: 1.6321

> Next epoch
Epoch 4/21
Val Epoch 4/21
Epoch: 4/20............. Train Loss: 1.5520 Val Loss: 1.5707

> Next epoch
Epoch 5/21
Val Epoch 5/21
Epoch: 5/20............. Train Loss: 1.4809 Val Loss: 1.5309

> Next epoch
Epoch 6/21
Val Epoch 6/21
Epoch: 6/20............. Train Loss: 1.4280 Val Loss: 1.5112

> Next epoch
Epoch 7/21
Val Epoch 7/21
Epoch: 7/20............. Train Loss: 1.3856 Val Loss: 1.5005

> Next epoch
Epoch 8/21
Val Epoch 8/21
Epoch: 8/20............. Train Loss: 1.3482 Val Loss: 1.4983

> Next epoch
Epoch 9/21
Val Epoch 9/21
Epoch: 9/20............. Train Loss: 1.3155 Val Loss: 1.5026

> Next epoch
Epoch 10/21
Val Epoch 10/21
Epoch: 10/20............. Train Loss: 1.2849 Val Loss: 1.5150



### Predicting the input sequences


In [26]:
def sample_from_probs(probs, top_n=10):
    """
    truncated weighted random choice.
    """
    _, indices = torch.sort(probs)
    # set probabilities after top_n to 0
    probs[indices.data[:-top_n]] = 0
    #print(probs.shape)
    sampled_index = torch.multinomial(probs, 1)
    return sampled_index

def predict_probs(model, hidden, character, vocab):
    # One-hot encoding our input to fit into the model
    character = np.array([[vocab.char2int[c] for c in character]])
    #character = one_hot_encode(character, len(vocab.char2int), character.shape[1], 1)
    character = one_hot_encode(character, model.vocab_size)
    character = torch.from_numpy(character)
    character = character.to(device)
    
    out, hidden = model(character, hidden)

    prob = nn.functional.softmax(out[-1], dim=0).data

    return prob, hidden

Let’s test our model now and see what kind of output we will get.

In [27]:
def generate_from_text(model, out_len, vocab, top_n=1, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Generate the initial hidden state
    state = model.init_state(device, 1)
    
    # Warm up the initial state, predicting on the initial string
    for ch in chars:
        #char, state = predict(model, ch, state, top_n=top_k)
        probs, state = predict_probs(model, state, ch, vocab)
        next_index = sample_from_probs(probs, top_n)

    # Now pass in the previous characters and get a new one
    for ii in range(size):
        #char, h = predict_char(model, chars, vocab)
        probs, state = predict_probs(model, state, chars, vocab)
        next_index = sample_from_probs(probs, top_n)
        # append to sequence
        chars.append(vocab.int2char[next_index.data[0]])

    return ''.join(chars)

In [29]:
text_predicted = generate_from_text(model, 1000, vocab, 3, 'we want ')
print(text_predicted)
print(len(text_predicted))

we want our dear lord. that, i am confession. and shall we hear thou wint, and friends are might. go, but thou speak'st true, i am not some offices of this amorous to your father as they than fellow. this is my loving love and leave of this. this is a blows of blood and heart's lord. why should i death? would not be satisfied; and now i stay, i stay; for i will stir a son, they have befull'd by himself to-day. why, that's a finger than the sine all through the duke of norfolk and hereford, as thy bed with me, and therefore his son whiles i have to day. i'll tell upon their paward. to change but the missearen deeds may speak to me. i will not bear my power with him. but she will be revolged to this subject and this soul so shrink and father you. if you will nay, but but to lien so noble. there is no blook, she shall be thus. therefore, make the like, and there begins against the duke of gloucester and their fathers. and they are framed in her. this, when have been togation that homours 

In [31]:

def generate_from_char(model, out_len, vocab, top_n=1, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Generate the initial hidden state
    state = model.init_state(device, 1)
    # Warm up the initial state, predicting on the initial string
    for ch in chars:
        #char, state = predict(model, ch, state, top_n=top_k)
        probs, state = predict_probs(model, state, ch, vocab)
        next_index = sample_from_probs(probs, top_n)
        
    # Include the last char predicted to the predicted output
    chars.append(vocab.int2char[next_index.data[0]])   
    
    # Now pass in the previous characters and get a new one
    for ii in range(size-1):
        #char, h = predict_char(model, chars, vocab)
        probs, state = predict_probs(model, state, chars[-1], vocab)
        next_index = sample_from_probs(probs, top_n)
        # append to sequence
        chars.append(vocab.int2char[next_index.data[0]])

    return ''.join(chars)

In [32]:
text_predicted = generate_from_char(model, 1000, vocab, 3, 'we want ')
print(text_predicted)
print(len(text_predicted))

we want and darknest that himself, i here better it was a poison, thou hast a swoed with me, and true and takest me affect thy death. why, have you done, for this six indeed. and in the bitter craves of this strangerns not the believe that the staff, the lass i holl, her brother chamber at. i chide thy helb, so meed thee, i am contrar. we are confess the stroke, and to be so forbids the tome. what's the defect should have their souls, to do thee, for i have been beating endsers than the such and they have desperatedent would build him with a purse and the botherood of thy father so let his hard, and will i live to take an humbly in the bold of incuse more men. i'll take your hearts to see you this; and, for this capital cousin of as said i loare of secret beauty, and not take it in his former father, by the heavens that sending of the bourness of his cousen of my country's foul in hollion, to right whiles in this regur throwe to bear a stirrung and an issue, shalt thou tell my strange 

### Conclusion

The model is now better as compared to the previous model after changing some model parameters though it end up training for long.